In [1]:
import torch
from torch_geometric.data import Data

In [2]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [3]:
data

Data(edge_index=[2, 4], x=[3, 1])

In [4]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index.t().contiguous())

In [5]:
data

Data(edge_index=[2, 4], x=[3, 1])

In [6]:
print(data.keys)

['x', 'edge_index']


## Dataset

In [7]:
from torch_geometric.datasets import TUDataset

In [8]:
dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...
Done!


In [9]:
len(dataset)

600

In [11]:
dataset[0]

Data(edge_index=[2, 168], x=[37, 3], y=[1])

## Mini Batch

In [13]:
from torch_geometric.data import DataLoader
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [15]:
for batch in loader:
    print(batch)
    print(batch.num_graphs)

Batch(batch=[1072], edge_index=[2, 4228], x=[1072, 3], y=[32])
32
Batch(batch=[977], edge_index=[2, 3728], x=[977, 3], y=[32])
32
Batch(batch=[1017], edge_index=[2, 3964], x=[1017, 3], y=[32])
32
Batch(batch=[1037], edge_index=[2, 4000], x=[1037, 3], y=[32])
32
Batch(batch=[1086], edge_index=[2, 4142], x=[1086, 3], y=[32])
32
Batch(batch=[1070], edge_index=[2, 4308], x=[1070, 3], y=[32])
32
Batch(batch=[957], edge_index=[2, 3690], x=[957, 3], y=[32])
32
Batch(batch=[1048], edge_index=[2, 4130], x=[1048, 3], y=[32])
32
Batch(batch=[1016], edge_index=[2, 3962], x=[1016, 3], y=[32])
32
Batch(batch=[1153], edge_index=[2, 4338], x=[1153, 3], y=[32])
32
Batch(batch=[1058], edge_index=[2, 4012], x=[1058, 3], y=[32])
32
Batch(batch=[903], edge_index=[2, 3480], x=[903, 3], y=[32])
32
Batch(batch=[945], edge_index=[2, 3352], x=[945, 3], y=[32])
32
Batch(batch=[1132], edge_index=[2, 4298], x=[1132, 3], y=[32])
32
Batch(batch=[1006], edge_index=[2, 3916], x=[1006, 3], y=[32])
32
Batch(batch=[1016]

## Learning

In [16]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [20]:
dataset[0]

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

In [17]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

cuda


In [19]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8160
